In [1]:
import pandas as pd
import numpy as np
import sklearn
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
from sklearn.ensemble import ExtraTreesRegressor
import joblib

In [2]:
df = pd.read_csv('data/DJIA_20140831_20250220.csv')

df["TradingDate"] = pd.to_datetime(df["TradingDate"])
df.sort_values(by="TradingDate",inplace=True)

In [3]:
components = ['HON', 'PFE', 'NKE', 'CVX', 'GS', 'JNJ', 'AAPL', 'HD', 'MSFT', 'VZ', 'CSCO', 'AXP', 'AMGN', 'MRK', 'JPM', 'WMT', 'CRM', 'NVDA', 'V', 'CAT', 'DIS', 'MMM', 'SHW',
'BA', 'PG', 'KO','UNH', 'MCD', 'TRV', 'WBA', 'AMZN', 'IBM', 'INTC', 'DOW']

In [3]:
df = df.rename(columns={'ClosePrice': 'close', 'OpenPrice': 'open', 'HighPrice': 'high', 'LowPrice': 'low', 'Amount': 'volume'})

In [4]:
def get_stock_data(df, symbol, end, start="2014-08-31"):
    data = df[(df["Symbol"] == symbol)& (df["TradingDate"] >= start) & (df["TradingDate"] <= end)]
    return data

def add_technical_indicators(df_):

    df = df_.copy()
    # Log Returns (using close price)
    df.loc[:, 'Log_Return'] = np.log(df['close'] / df['close'].shift(1))
    
    # High-Low Range
    df.loc[:, 'High_Low_Range'] = df['high'] - df['low']
    
    # Close-to-Open Return
    df.loc[:, 'Close_to_Open_Return'] = (df['close'] - df['open']) / df['open']
    
    # Close-to-High and Close-to-Low ratios
    df.loc[:, 'Close_to_High'] = (df['close'] - df['low']) / (df['high'] - df['low'])
    df.loc[:, 'Close_to_Low'] = (df['high'] - df['close']) / (df['high'] - df['low'])
    
    # Volume Change
    df.loc[:, 'Volume_Change'] = (df['volume'] - df['volume'].shift(1)) / df['volume'].shift(1)
    
    
    # Daily Volatility (Rolling Standard Deviation of Log Return over 14 days)
    df.loc[:, 'Volatility'] = df['Log_Return'].rolling(window=14).std()
    
    # Rate of Change (ROC) over 14 days
    df.loc[:, 'ROC'] = (df['close'] - df['close'].shift(14)) / df['close'].shift(14)
    
    # Rolling Mean of Close Price over 14 days
    df.loc[:, 'Rolling_Mean'] = df['close'].rolling(window=14).mean()
    
    # Rolling Std Dev of Close Price over 14 days
    df.loc[:, 'Rolling_Std_Dev'] = df['close'].rolling(window=14).std()

    # ROCP (Rate of Change Percentage)

    df.loc[:, 'ROCP_close'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
    df.loc[:, 'ROCP_open'] = (df['open'] - df['open'].shift(1)) / df['open'].shift(1)
    df.loc[:, 'ROCP_high'] = (df['high'] - df['high'].shift(1)) / df['high'].shift(1)
    df.loc[:, 'ROCP_low'] = (df['low'] - df['low'].shift(1)) / df['low'].shift(1)

    # Moving Average (MA)
    df.loc[:, 'MA'] = df['close'].rolling(window=14).mean()  # Example for 14-period moving average

    # MACP (Moving Average Change Percentage)
    df.loc[:, 'MACP'] = df['MA'] - df['close']

    # MACD (Moving Average Convergence/Divergence)
    Nfast = 12  # Default fast EMA period
    Nslow = 26  # Default slow EMA period
    df.loc[:, 'EMA_fast'] = df['close'].ewm(span=Nfast, adjust=False).mean()
    df.loc[:, 'EMA_slow'] = df['close'].ewm(span=Nslow, adjust=False).mean()
    df.loc[:, 'DIF'] = df['EMA_fast'] - df['EMA_slow']
    df.loc[:, 'DEA'] = df['DIF'].ewm(span=9, adjust=False).mean()  # Default smoothing constant
    df.loc[:, 'MACD'] = 2 * (df['DIF'] - df['DEA'])

    # RSI (Relative Strength Index)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    RS = gain / loss
    df.loc[:, 'RSI'] = 100 - (100 / (1 + RS))

    # VROCP (Volume Rate of Change Percentage)
    df.loc[:, 'VROCP'] = (df['volume'] - df['volume'].shift(1)) / df['volume'].shift(1)

    # VROC (Volume Rate of Change)
    df.loc[:, 'VROC'] = (df['volume'] - df['volume'].shift(1))

    # Bollinger Bands
    df.loc[:, 'BOLL_Middle'] = df['close'].rolling(window=14).mean()  # Middle Band = MA
    df.loc[:, 'BOLL_STD'] = df['close'].rolling(window=14).std()  # Standard deviation
    df.loc[:, 'BOLL_upper'] = df['BOLL_Middle'] + (2 * df['BOLL_STD'])  # Upper Band
    df.loc[:, 'BOLL_lower'] = df['BOLL_Middle'] - (2 * df['BOLL_STD'])  # Lower Band

     # Price-to-Volume Correlation (Rolling 14 days)
    df.loc[:, 'Price_Volume_Correlation'] = df['close'].rolling(window=14).corr(df['volume'])

    df = df.dropna()
    return df



In [5]:
def construct_training_data(df, h=6):
    
    # Shift the close price by 'h' days to get the target
    df['target'] = df['close'].shift(-h)  # Target is the close price 'h' days later

    # Drop rows with NaN values in the target column or technical indicators
    df = df.dropna()

    y = df['target']  # Target: Close price 'h' days later
    X = df.drop(columns=['target', 'Symbol'])  # Features
    return X, y

def construct_test_data(df, h=6):
    
    # Shift the close price by 'h' days to get the target
    df['target'] = df['close'].shift(-h)  # Target is the close price 'h' days later

    # Drop rows with NaN values in the target column or technical indicators
    df = df.dropna()

    y = df['target', 'TradingDate']  # Target: Close price 'h' days later
    X = df.drop(columns=['target', 'Symbol'])  # Features
    return X, y

In [7]:
# training

models_dict_6 = {}
for symbol in components:
    print(f'Processing {symbol}')
    data = get_stock_data(df, symbol, '2020-08-30')
    data = add_technical_indicators(data)
    X, y = construct_training_data(data)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    X_train = X_train.drop(columns=["TradingDate"])
    X_train = X_train.values
    y_train = y_train.values

    print(X_train.shape, y_train.shape)
    
    # Initialize TPOT Regressor
    tpot_ = TPOTRegressor(generations=15, population_size=40, verbosity=2, random_state=42, n_jobs=-1)

    # Fit the TPOT Regressor
    tpot_.fit(X_train, y_train)

    # Store the model in a dictionary (could be for multiple problems)
    models_dict_6[symbol] = tpot_.fitted_pipeline_
    # print(f'Exporting model for {symbol}')
    print('*'*50)

Processing HON
(1117, 39) (1117,)


Version 0.12.2 of tpot is outdated. Version 1.0.0 was released 3 days ago.


Optimization Progress:   0%|          | 0/640 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -11.815511302161733

Generation 2 - Current best internal CV score: -11.488607624777824

Generation 3 - Current best internal CV score: -11.48801934901163

Generation 4 - Current best internal CV score: -11.188819722526295

Generation 5 - Current best internal CV score: -11.188819722526295

Generation 6 - Current best internal CV score: -11.182109212171948

Generation 7 - Current best internal CV score: -9.877142760398243

Generation 8 - Current best internal CV score: -9.542114638032718

Generation 9 - Current best internal CV score: -9.150974290067472

Generation 10 - Current best internal CV score: -9.150974290067472

Generation 11 - Current best internal CV score: -9.150974290067472

Generation 12 - Current best internal CV score: -9.150974290067472

Generation 13 - Current best internal CV score: -8.342558498062626

Generation 14 - Current best internal CV score: -8.342558498062626

Generation 15 - Current best internal CV score: -8.

Version 0.12.2 of tpot is outdated. Version 1.0.0 was released 3 days ago.


...

Optimization Progress:   0%|          | 0/640 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -2.4216025859887718

Generation 2 - Current best internal CV score: -2.4216025859887718

Generation 3 - Current best internal CV score: -2.4216025859887718

Generation 4 - Current best internal CV score: -2.195509543487705

Generation 5 - Current best internal CV score: -2.1631366189421497

Generation 6 - Current best internal CV score: -2.1631366189421497

Generation 7 - Current best internal CV score: -2.1631366189421497

Generation 8 - Current best internal CV score: -2.1213035894709598

Generation 9 - Current best internal CV score: -2.0885243178606308

Generation 10 - Current best internal CV score: -2.0870989655460583

Generation 11 - Current best internal CV score: -1.9308252516554247

Generation 12 - Current best internal CV score: -1.9308252516554247

Generation 13 - Current best internal CV score: -1.9308252516554247

Generation 14 - Current best internal CV score: -1.9308252516554247

Generation 15 - Current best internal CV sc

Version 0.12.2 of tpot is outdated. Version 1.0.0 was released 4 days ago.


Optimization Progress:   0%|          | 0/640 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -4.489999530207965

Generation 2 - Current best internal CV score: -4.489999530207965

Generation 3 - Current best internal CV score: -4.489999530207965

Generation 4 - Current best internal CV score: -3.9829539901440336

Generation 5 - Current best internal CV score: -3.9829539901440336

Generation 6 - Current best internal CV score: -3.9749185674396577

Generation 7 - Current best internal CV score: -3.972145953081255

Generation 8 - Current best internal CV score: -3.7985593282993806

Generation 9 - Current best internal CV score: -3.490150994947199

Generation 10 - Current best internal CV score: -3.490150994947199

Generation 11 - Current best internal CV score: -3.4209086259773733

Generation 12 - Current best internal CV score: -3.412733640617631

Generation 13 - Current best internal CV score: -3.256441590139887

Generation 14 - Current best internal CV score: -3.256441590139887

Generation 15 - Current best internal CV score: -3.

In [8]:
models_dict_12 = {}
for symbol in components:
    print(f'Processing {symbol}')
    data = get_stock_data(df, symbol, '2020-08-30')
    data = add_technical_indicators(data)
    X, y = construct_training_data(data, h=12)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train = X_train.drop(columns=["TradingDate"])

    X_train = X_train.values
    y_train = y_train.values

    X_test = X_test.drop(columns=["TradingDate"])
    # Initialize TPOT Regressor
    tpot_ = TPOTRegressor(generations=15, population_size=40, verbosity=2, random_state=42, n_jobs=-1, early_stop=5)

    # Fit the TPOT Regressor
    tpot_.fit(X_train, y_train)

    # Store the model in a dictionary (could be for multiple problems)
    models_dict_12[symbol] = tpot_.fitted_pipeline_
    # print(f'Exporting model for {symbol}')
    print('*'*50)

Processing HON


Version 0.12.2 of tpot is outdated. Version 1.0.0 was released 4 days ago.


Optimization Progress:   0%|          | 0/640 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -13.591507202731446

Generation 2 - Current best internal CV score: -13.591507202731446

Generation 3 - Current best internal CV score: -13.591507202731446

Generation 4 - Current best internal CV score: -13.236070724743616

Generation 5 - Current best internal CV score: -12.482468458931669

Generation 6 - Current best internal CV score: -12.482468458931669

Generation 7 - Current best internal CV score: -12.482468458931669

Generation 8 - Current best internal CV score: -12.1811744957403

Generation 9 - Current best internal CV score: -12.1811744957403

Generation 10 - Current best internal CV score: -11.902773270748298

Generation 11 - Current best internal CV score: -11.902773270748298

Generation 12 - Current best internal CV score: -10.797436001440268

Generation 13 - Current best internal CV score: -10.797436001440268

Generation 14 - Current best internal CV score: -10.797436001440268

Generation 15 - Current best internal CV score

Version 0.12.2 of tpot is outdated. Version 1.0.0 was released 4 days ago.


...

Optimization Progress:   0%|          | 0/640 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -3.3206294962816356

Generation 2 - Current best internal CV score: -3.2564405279500193

Generation 3 - Current best internal CV score: -3.2506311768282528

Generation 4 - Current best internal CV score: -3.114723243221436

Generation 5 - Current best internal CV score: -3.07263191284744

Generation 6 - Current best internal CV score: -2.9460274046328614

Generation 7 - Current best internal CV score: -2.9435374403065664

Generation 8 - Current best internal CV score: -2.6478487146343115

Generation 9 - Current best internal CV score: -2.6478487146343115

Generation 10 - Current best internal CV score: -2.6478487146343115

Generation 11 - Current best internal CV score: -2.6478487146343115

Generation 12 - Current best internal CV score: -2.6478487146343115

Generation 13 - Current best internal CV score: -2.6478487146343115

Generation 14 - Current best internal CV score: -2.6478487146343115

Generation 15 - Current best internal CV scor

Version 0.12.2 of tpot is outdated. Version 1.0.0 was released 4 days ago.


Optimization Progress:   0%|          | 0/640 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -8.545765150317214

Generation 2 - Current best internal CV score: -7.814819128025308

Generation 3 - Current best internal CV score: -7.814819128025308

Generation 4 - Current best internal CV score: -7.21822821577089

Generation 5 - Current best internal CV score: -6.7711947770629255

Generation 6 - Current best internal CV score: -6.668888419799629

Generation 7 - Current best internal CV score: -6.668888419799629

Generation 8 - Current best internal CV score: -6.668888419799629

Generation 9 - Current best internal CV score: -6.2380882469414765

Generation 10 - Current best internal CV score: -6.2380882469414765

Generation 11 - Current best internal CV score: -6.2380882469414765

Generation 12 - Current best internal CV score: -5.5504728027748165

Generation 13 - Current best internal CV score: -5.5504728027748165

Generation 14 - Current best internal CV score: -5.5504728027748165

Generation 15 - Current best internal CV score: -5

In [9]:
# Save the model_dict
with open('model_dict.joblib_12', 'wb') as f:
    joblib.dump(models_dict_12, f)

In [18]:
DJI = pd.read_csv('data/DJIA/DJIA.csv') # from 2014-01-02 to 2025-02-20
DJI.rename(columns={'Indexcd':'Symbol', 'Opnidx': 'open', 'Highidx': 'high', 'Lowidx': 'low', 'Clsidx': 'close','Vol': 'volume', 'Trddt': 'TradingDate', 'Value': 'return'}, inplace=True)

DJI["TradingDate"] = pd.to_datetime(DJI["TradingDate"])
DJI.sort_values(by="TradingDate",inplace=True)

data = DJI[(DJI["TradingDate"] <= "2020-08-30")]

data = add_technical_indicators(data)
X, y = construct_training_data(data)
X_train, y_train = X.drop(columns=["TradingDate"]).values, y.values

print(X_train.shape, y_train.shape)

tpot_ = TPOTRegressor(generations=20, population_size=50, verbosity=2, random_state=42, n_jobs=-1)
tpot_.fit(X_train, y_train)

models_dict_6["DJI"] = tpot_.fitted_pipeline_

(1658, 35) (1658,)


Version 0.12.2 of tpot is outdated. Version 1.0.0 was released 4 days ago.


Optimization Progress:   0%|          | 0/1050 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -363518.33759396354

Generation 2 - Current best internal CV score: -361718.48602301336

Generation 3 - Current best internal CV score: -360939.08206406917

Generation 4 - Current best internal CV score: -360818.5608347687

Generation 5 - Current best internal CV score: -360818.5608347687

Generation 6 - Current best internal CV score: -360818.5608347687

Generation 7 - Current best internal CV score: -360818.5608347687

Generation 8 - Current best internal CV score: -360818.5608347687

Generation 9 - Current best internal CV score: -360818.5608347687

Generation 10 - Current best internal CV score: -360818.5608347687

Generation 11 - Current best internal CV score: -360818.56082503527

Generation 12 - Current best internal CV score: -356749.87506519293

Generation 13 - Current best internal CV score: -356749.87506519293

Generation 14 - Current best internal CV score: -356749.87506519293

Generation 15 - Current best internal CV score: -

In [19]:
X, y = construct_training_data(data, h=12)

X_train, y_train = X.drop(columns=["TradingDate"]).values, y.values

tpot_ = TPOTRegressor(generations=20, population_size=50, verbosity=2, random_state=42, n_jobs=-1)
tpot_.fit(X_train, y_train)

models_dict_12["DJI"] = tpot_.fitted_pipeline_

Version 0.12.2 of tpot is outdated. Version 1.0.0 was released 4 days ago.


Optimization Progress:   0%|          | 0/1050 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -737551.1064665222

Generation 2 - Current best internal CV score: -725526.2386303778

Generation 3 - Current best internal CV score: -725526.2386303778

Generation 4 - Current best internal CV score: -724194.1591746692

Generation 5 - Current best internal CV score: -724194.1591320242

Generation 6 - Current best internal CV score: -724194.1591319825

Generation 7 - Current best internal CV score: -724191.3271399911

Generation 8 - Current best internal CV score: -724191.3271399911

Generation 9 - Current best internal CV score: -724191.3271399911

Generation 10 - Current best internal CV score: -724191.3271399911

Generation 11 - Current best internal CV score: -724191.2988038177

Generation 12 - Current best internal CV score: -724191.2988034335

Generation 13 - Current best internal CV score: -723856.4120204169

Generation 14 - Current best internal CV score: -723856.4120204169

Generation 15 - Current best internal CV score: -723854.

In [20]:
# Save the model_dict
with open('model/model_dict.joblib_12', 'wb') as f:
    joblib.dump(models_dict_12, f)

with open('model/model_dict.joblib_6', 'wb') as f:
    joblib.dump(models_dict_6, f)

In [6]:
models_dict_6 = joblib.load('model/model_dict.joblib_6')
models_dict_12 = joblib.load('model/model_dict.joblib_12')